In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [20]:
url = "https://www.melon.com/chart/week/index.htm"
res = requests.get(url, headers = {'User-Agent':'Mozilla/5.0'})
raw = res.text
soup = BeautifulSoup(raw,'html.parser')

In [8]:
type(soup)

bs4.BeautifulSoup

In [18]:
date = soup.select_one('span.yyyymmdd')
date = date.text.strip('\n\t\r')
date

'2020.11.09 ~ 2020.11.15'

In [23]:
startday = date[:10]
endday = date[13:]
startday, endday

('2020.11.09', '2020.11.15')

In [72]:
service = soup.select_one('.service_list_song')
tbody = service.select_one('tbody')
trs = tbody.select('tr')
len(trs)

100

In [113]:
for i in [0,1,2,19]:
    entry = trs[i].select_one('.rank_wrap')
    spans = entry.find_all('span')
    # 신규 진입이면 2, 저번주에 있었으면 3
    if len(spans) == 3:
        print(spans[2].text)
        if diff != 0:
            # class태그의 값도 꺼내는군. 처음본다
            sign = spans[2].attrs['class']
            if sign[0] == 'down':
                diff *= -1
    else: 
        print('신규진입')

0
1
1
신규진입


In [152]:
print(trs[0].select('.ellipsis')[0].text)
print(trs[0].select('.ellipsis')[1].select_one('a').text)
print(trs[0].select('.ellipsis')[2].text)


Dynamite

방탄소년단

Dynamite (DayTime Version)



In [212]:
ranks = [] ; updowns = [] ; nums = [] ; titles = [] ; singers = [] ; albums = [] 

for tr in trs:
    rank = tr.select_one('span.rank').text
    # print(rank)
    updown = tr.select_one('.rank_wrap').text.strip('\n\t\r')[3:5]
    num = tr.select_one('.rank_wrap').text.strip('\n\t\r')[6:]

    
    title = tr.select('.ellipsis')[0].text.strip(' \n\t\r')
    singer = tr.select('.ellipsis')[1].select_one('a').text.strip(' \n\t\r')
    album = tr.select('.ellipsis')[2].text.strip(' \n\t\r')
    # cnt = tr.select_one('span.cnt > span')
    # 이거 왜 숫자가 안 들어오지?
    # => 동적인가봐, beautifulsoup으로는 못가져와
    # 나중에 selenium에서 xmlpath로 가져와야 함

    ranks.append(rank)
    updowns.append(updown)
    nums.append(num)
    titles.append(title)
    singers.append(singer)
    albums.append(album)




    # print(title,singer,album,'\n',cnt)
    

In [238]:
top100 = pd.DataFrame({
    '순위': ranks,
    '전주대비': updowns,
    '변동': nums,
    '제목': titles,
    '가수': singers,
    '앨범': albums
})
top100.set_index('순위',inplace=True)
print(f'\t\t\t\t\t{startday} ~ {endday}')
top100

					2020.11.09 ~ 2020.11.15


,전주대비,변동,제목,가수,앨범
순위,,,,,
1,동일,0,Dynamite,방탄소년단,Dynamite (DayTime Version)
2,상승,1,힘든 건 사랑이 아니다,임창정,힘든 건 사랑이 아니다
3,하락,1,Lovesick Girls,BLACKPINK,THE ALBUM
4,상승,1,취기를 빌려 (취향저격 그녀 X 산들),산들,취기를 빌려 (취향저격 그녀 X 산들)
5,하락,1,DON'T TOUCH ME,환불원정대,DON'T TOUCH ME
...,...,...,...,...,...
96,하락,4,화려하지 않은 고백,규현 (KYUHYUN),슬기로운 의사생활 OST Part 4
97,진입,,MAGO,여자친구 (GFRIEND),回:Walpurgis Night
98,하락,3,행복하니,케이시 (Kassy),추(秋)억


In [228]:
fluctuation = top100.pivot('제목','전주대비','변동')
fluctuation

전주대비,동일,상승,진입,하락
제목,,,,
12:45 (Stripped),NaN,NaN,NaN,1
2002,0,NaN,NaN,NaN
AYA,NaN,28,NaN,NaN
Bad Boy,NaN,NaN,NaN,2
Blueming,NaN,NaN,NaN,2
...,...,...,...,...
행복해,NaN,8,NaN,NaN
홀로,NaN,NaN,NaN,2
화려하지 않은 고백,NaN,NaN,NaN,4


In [229]:
fluctuation['동일'][0]

nan

In [230]:
import numpy as np
fluctuation[fluctuation['동일'].notnull()][['동일']]

전주대비,동일
제목,
2002,0
Dolphin,0
Dynamite,0
How You Like That,0
너를 만나,0
늦은 밤 너의 집 앞 골목길에서,0
봄날,0
사랑이란 멜로는 없어,0
시든 꽃에 물을 주듯,0


In [245]:
up = fluctuation[fluctuation['상승'].notnull()][['상승']]
up.sort_values(by='상승', ascending = False)

전주대비,상승
제목,
도망가,8
행복해,8
별을 담은 시 (Ode To The Stars),7
HIP,4
흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야,4
나랑 같이 걸을래 (바른연애 길잡이 X 적재),4
이제 나만 믿어요,3
METEOR,3
딩가딩가 (Dingga),3


In [232]:
fluctuation[fluctuation['하락'].notnull()][['하락']]

전주대비,하락
제목,
12:45 (Stripped),1
Bad Boy,2
Blueming,2
DON'T TOUCH ME,1
Dance Monkey,2
Don't Start Now,3
Downtown Baby,3
Ice Cream (with Selena Gomez),8
Into the I-LAND,4


In [233]:
fluctuation[fluctuation['진입'].notnull()][['진입']]

전주대비,진입
제목,
MAGO,
My Love (Duet Ver.),
가을밤에 든 생각,
너의 하루 (바른연애 길잡이 X 로꼬),
니 이름 불렀나 봐,
